# Multi-Layer Perceptron (MLP)
This notebook builts an MLP for classification, same way as described in [Cepeda Humerez et al. (2019)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007290)

Hyperparameters to use:

````python
input_size = 200  # Adjust based on dataset
hidden_size = [300, 200] # 300 and 200 LTUs
output_size = 2  # Number of classes
dropout_rate = 0.5 # This wasn't specified in the paper, but choose any
learning_rate = 0.001 # Not specified in the paper
epochs = 100 # Not specified in the paper
batch_size = 16 # Not specified in the paper
````

The model architecture is in ``MLP.py``

Load the MLP model codes from ``src``

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tqdm
from sympy import sqrt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
# Import all the functions from the 'src' directory, we import all the functions from each module so we can use them straight away
from ssa_simulation import *
from ssa_analysis import *
from ssa_classification import *
from models.MLP import MLP 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Example usage to train an MLP

In [13]:
# Example usage
input_size = 200  # Adjust based on dataset
hidden_size = [300, 200]
output_size = 2  # Number of classes
dropout_rate = 0.5
learning_rate = 0.001
epochs = 100
batch_size = 16

# Generate synthetic data
X_train = torch.randn(1000, input_size)
y_train = torch.randint(0, output_size, (1000,))
X_val = torch.randn(200, input_size)
y_val = torch.randint(0, output_size, (200,))

# Convert to DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize and train model
file_path = "mlp_model.pth"
model = MLP(input_size, hidden_size, output_size, dropout_rate, learning_rate)
model.train_model(train_loader, val_loader, epochs, save_path=file_path)

# Load best model and evaluate
model.load_model(file_path)
test_acc = model.evaluate(val_loader)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Make predictions
X_test = torch.randn(5, input_size)
predictions = model.predict(X_test)
print("Predicted classes:", predictions)


🔄 Using device: cuda (1 GPUs available)
✅ Running on CUDA!
Epoch [1/100], Loss: 119.2893, Train Acc: 0.5050
Validation Acc: 0.5150
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.5150)
Epoch [2/100], Loss: 86.0712, Train Acc: 0.5620
Validation Acc: 0.4700
Epoch [3/100], Loss: 69.8308, Train Acc: 0.5800
Validation Acc: 0.4700
Epoch [4/100], Loss: 56.4978, Train Acc: 0.6110
Validation Acc: 0.5000
Epoch [5/100], Loss: 55.4314, Train Acc: 0.6290
Validation Acc: 0.5100
Epoch [6/100], Loss: 49.2156, Train Acc: 0.6370
Validation Acc: 0.4700
Epoch [7/100], Loss: 45.8456, Train Acc: 0.6470
Validation Acc: 0.4600
Epoch [8/100], Loss: 38.8641, Train Acc: 0.6860
Validation Acc: 0.4600
Epoch [9/100], Loss: 37.8630, Train Acc: 0.6940
Validation Acc: 0.4900
Epoch [10/100], Loss: 37.2601, Train Acc: 0.6940
Validation Acc: 0.5150
Epoch [11/100], Loss: 37.2800, Train Acc: 0.6990
Validation Acc: 0.4850
Epoch [12/100], Loss: 32.3891, Train Acc: 0.7630
Validation Acc: 0.5100
Epoch [13/100], Loss: 32

Train the MLP using SSA data, we need to first standardise the data. If we don't, the loss will be showing as nan, which is incorrect. 

In [26]:
# Train MLP model using SSA data
output_file = 'data/mRNA_trajectories_example.csv'
X_train, X_test, y_train, y_test = load_and_split_data(output_file)
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
# Standardize the data 
# If your input features are too large (e.g., >1000) or too small (<0.0001), it can cause unstable training, so it's better to standardize the data.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)

[[ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 ...
 [ 0.          1.00514142 -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.          2.44105774 -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]]


In [28]:
# Define model parameters
input_size = X_train.shape[1]
output_size = len(set(y_train))  # Number of classes
hidden_size = [300, 200]
dropout_rate = 0.5
learning_rate = 0.001
epochs = 100
batch_size = 32

train_loader = DataLoader(TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.long)),
    batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.long)),
    batch_size=batch_size, shuffle=False
)

model = MLP(input_size, hidden_size, output_size, dropout_rate, learning_rate)
model.train_model(train_loader, epochs=epochs)

# Evaluate MLP model
mlp_accuracy = model.evaluate(test_loader)
print(f"MLP Test Accuracy: {mlp_accuracy:.4f}")

🔄 Using device: cuda (1 GPUs available)
✅ Running on CUDA!
Epoch [1/100], Loss: 11.6425, Train Acc: 0.6469
Epoch [2/100], Loss: 7.2031, Train Acc: 0.7094
Epoch [3/100], Loss: 7.4694, Train Acc: 0.7063
Epoch [4/100], Loss: 7.5236, Train Acc: 0.7531
Epoch [5/100], Loss: 5.8111, Train Acc: 0.7750
Epoch [6/100], Loss: 5.4871, Train Acc: 0.7844
Epoch [7/100], Loss: 5.2564, Train Acc: 0.7969
Epoch [8/100], Loss: 5.1997, Train Acc: 0.7781
Epoch [9/100], Loss: 5.7426, Train Acc: 0.7719
Epoch [10/100], Loss: 5.1096, Train Acc: 0.7531
Epoch [11/100], Loss: 4.6505, Train Acc: 0.7969
Epoch [12/100], Loss: 4.7923, Train Acc: 0.7906
Epoch [13/100], Loss: 5.5862, Train Acc: 0.7812
Epoch [14/100], Loss: 5.3528, Train Acc: 0.7656
Epoch [15/100], Loss: 5.0964, Train Acc: 0.7625
Epoch [16/100], Loss: 5.2844, Train Acc: 0.7906
Epoch [17/100], Loss: 4.4309, Train Acc: 0.8031
Epoch [18/100], Loss: 4.9868, Train Acc: 0.7937
Epoch [19/100], Loss: 4.7583, Train Acc: 0.7875
Epoch [20/100], Loss: 4.4636, Train A

Same as above, but getting the accuracy in a one-liner

In [32]:
# Train SVM model using SSA data
output_file = 'data/mRNA_trajectories_example.csv'
X_train, X_test, y_train, y_test = load_and_split_data(output_file)
mlp_accuracy = mlp_classifier(X_train, X_test, y_train, y_test, epochs=100)

🔄 Using device: cuda (1 GPUs available)
✅ Running on CUDA!
Epoch [1/100], Loss: 8.6367, Train Acc: 0.6500
Epoch [2/100], Loss: 5.9576, Train Acc: 0.7406
Epoch [3/100], Loss: 5.5829, Train Acc: 0.7688
Epoch [4/100], Loss: 4.6564, Train Acc: 0.7969
Epoch [5/100], Loss: 4.6200, Train Acc: 0.7875
Epoch [6/100], Loss: 4.4650, Train Acc: 0.8031
Epoch [7/100], Loss: 4.6749, Train Acc: 0.7875
Epoch [8/100], Loss: 4.5802, Train Acc: 0.7906
Epoch [9/100], Loss: 4.6727, Train Acc: 0.7969
Epoch [10/100], Loss: 4.3787, Train Acc: 0.7875
Epoch [11/100], Loss: 4.4203, Train Acc: 0.8219
Epoch [12/100], Loss: 4.4238, Train Acc: 0.7969
Epoch [13/100], Loss: 4.1009, Train Acc: 0.8031
Epoch [14/100], Loss: 4.4463, Train Acc: 0.7937
Epoch [15/100], Loss: 4.2769, Train Acc: 0.8000
Epoch [16/100], Loss: 4.0623, Train Acc: 0.8031
Epoch [17/100], Loss: 4.1207, Train Acc: 0.8000
Epoch [18/100], Loss: 4.1053, Train Acc: 0.8031
Epoch [19/100], Loss: 3.8896, Train Acc: 0.8063
Epoch [20/100], Loss: 4.0101, Train Ac